In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 256
tree_depth = 10
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.164484977722168 | KNN Loss: 6.233161449432373 | BCE Loss: 1.931323766708374
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.180647850036621 | KNN Loss: 6.2330732345581055 | BCE Loss: 1.9475743770599365
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.156098365783691 | KNN Loss: 6.233052730560303 | BCE Loss: 1.9230459928512573
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.168828964233398 | KNN Loss: 6.232822418212891 | BCE Loss: 1.936007022857666
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.191194534301758 | KNN Loss: 6.232855319976807 | BCE Loss: 1.9583394527435303
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.178317070007324 | KNN Loss: 6.232844352722168 | BCE Loss: 1.9454725980758667
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.167888641357422 | KNN Loss: 6.2326531410217285 | BCE Loss: 1.935235619544983
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.10910415649414 | KNN Loss: 6.23281192779541 | BCE Loss: 1.87629258

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.361576080322266 | KNN Loss: 6.212650775909424 | BCE Loss: 1.148925542831421
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.327655792236328 | KNN Loss: 6.208916664123535 | BCE Loss: 1.118739128112793
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.329785346984863 | KNN Loss: 6.207803249359131 | BCE Loss: 1.1219818592071533
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 7.311932563781738 | KNN Loss: 6.206353664398193 | BCE Loss: 1.105578899383545
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 7.303010940551758 | KNN Loss: 6.206175327301025 | BCE Loss: 1.0968353748321533
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 7.301403045654297 | KNN Loss: 6.20162296295166 | BCE Loss: 1.0997802019119263
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 7.295412540435791 | KNN Loss: 6.200352191925049 | BCE Loss: 1.0950604677200317
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 7.2853102684021 | KNN Loss: 6.198827266693115 | BCE Loss: 1.08

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.777968406677246 | KNN Loss: 5.711792945861816 | BCE Loss: 1.0661754608154297
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.72640323638916 | KNN Loss: 5.690771579742432 | BCE Loss: 1.0356318950653076
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.781209945678711 | KNN Loss: 5.708673000335693 | BCE Loss: 1.0725367069244385
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.753395080566406 | KNN Loss: 5.69069766998291 | BCE Loss: 1.0626976490020752
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.7406487464904785 | KNN Loss: 5.6938862800598145 | BCE Loss: 1.0467623472213745
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.773418426513672 | KNN Loss: 5.704648971557617 | BCE Loss: 1.0687696933746338
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.787770748138428 | KNN Loss: 5.717114448547363 | BCE Loss: 1.0706562995910645
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.706891059875488 | KNN Loss: 5.66427755355835 | BCE Loss

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.650391101837158 | KNN Loss: 5.624721050262451 | BCE Loss: 1.0256701707839966
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.719789028167725 | KNN Loss: 5.673291206359863 | BCE Loss: 1.0464977025985718
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.676051139831543 | KNN Loss: 5.617538928985596 | BCE Loss: 1.0585122108459473
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.666265487670898 | KNN Loss: 5.619668483734131 | BCE Loss: 1.046596884727478
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.716798782348633 | KNN Loss: 5.635405540466309 | BCE Loss: 1.0813934803009033
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.654577732086182 | KNN Loss: 5.638788223266602 | BCE Loss: 1.01578950881958
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.737710952758789 | KNN Loss: 5.6912736892700195 | BCE Loss: 1.046437382698059
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.695381164550781 | KNN Loss: 5.65086555480957 | BCE Loss: 1

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.675329208374023 | KNN Loss: 5.626804828643799 | BCE Loss: 1.0485246181488037
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.788908958435059 | KNN Loss: 5.719067096710205 | BCE Loss: 1.0698418617248535
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.6926984786987305 | KNN Loss: 5.659687042236328 | BCE Loss: 1.0330116748809814
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.709036350250244 | KNN Loss: 5.644835472106934 | BCE Loss: 1.0642008781433105
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.69319486618042 | KNN Loss: 5.628274917602539 | BCE Loss: 1.0649198293685913
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.7110772132873535 | KNN Loss: 5.625705242156982 | BCE Loss: 1.085371971130371
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.660208225250244 | KNN Loss: 5.618839740753174 | BCE Loss: 1.0413684844970703
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 6.687494277954102 | KNN Loss: 5.6368865966796875 | BCE Lo

...

Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 6.693940162658691 | KNN Loss: 5.643997669219971 | BCE Loss: 1.0499427318572998
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 6.675162315368652 | KNN Loss: 5.601166248321533 | BCE Loss: 1.07399582862854
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 6.631659984588623 | KNN Loss: 5.604273319244385 | BCE Loss: 1.0273866653442383
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.649306297302246 | KNN Loss: 5.607953071594238 | BCE Loss: 1.0413532257080078
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.697667598724365 | KNN Loss: 5.652248859405518 | BCE Loss: 1.0454188585281372
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.626442909240723 | KNN Loss: 5.607489585876465 | BCE Loss: 1.0189530849456787
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.717464447021484 | KNN Loss: 5.687590599060059 | BCE Loss: 1.0298739671707153
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 6.659090042114258 | KNN Loss: 5.6447601318359375 | B

Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 6.695256233215332 | KNN Loss: 5.636814117431641 | BCE Loss: 1.0584423542022705
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 6.671474933624268 | KNN Loss: 5.605099678039551 | BCE Loss: 1.0663752555847168
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 6.645374774932861 | KNN Loss: 5.601078033447266 | BCE Loss: 1.0442967414855957
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.737398624420166 | KNN Loss: 5.681279182434082 | BCE Loss: 1.056119441986084
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.646589279174805 | KNN Loss: 5.604992389678955 | BCE Loss: 1.0415971279144287
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.741371154785156 | KNN Loss: 5.673711776733398 | BCE Loss: 1.067659616470337
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.6905717849731445 | KNN Loss: 5.645541667938232 | BCE Loss: 1.045029878616333
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.740664482116699 | KNN Loss: 5.651845932006836 | B

Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 6.6612348556518555 | KNN Loss: 5.609503746032715 | BCE Loss: 1.0517313480377197
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 6.639632225036621 | KNN Loss: 5.60650634765625 | BCE Loss: 1.0331257581710815
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 6.713498115539551 | KNN Loss: 5.690204620361328 | BCE Loss: 1.0232934951782227
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.689447402954102 | KNN Loss: 5.658040523529053 | BCE Loss: 1.031407117843628
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.685724258422852 | KNN Loss: 5.6364359855651855 | BCE Loss: 1.0492883920669556
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.699162483215332 | KNN Loss: 5.618096828460693 | BCE Loss: 1.0810654163360596
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.6565656661987305 | KNN Loss: 5.613607406616211 | BCE Loss: 1.0429580211639404
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.690430164337158 | KNN Loss: 5.645288944244385 |

Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 6.666149616241455 | KNN Loss: 5.60941743850708 | BCE Loss: 1.056732177734375
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 6.658149719238281 | KNN Loss: 5.624458312988281 | BCE Loss: 1.0336915254592896
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 6.652418613433838 | KNN Loss: 5.60536527633667 | BCE Loss: 1.0470534563064575
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.755138397216797 | KNN Loss: 5.71406364440918 | BCE Loss: 1.0410747528076172
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.677190780639648 | KNN Loss: 5.620389461517334 | BCE Loss: 1.0568015575408936
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.673486709594727 | KNN Loss: 5.606107234954834 | BCE Loss: 1.0673794746398926
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.66376256942749 | KNN Loss: 5.61820650100708 | BCE Loss: 1.0455561876296997
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.731047630310059 | KNN Loss: 5.666060447692871 | BCE Lo

Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 6.668487548828125 | KNN Loss: 5.611492156982422 | BCE Loss: 1.0569953918457031
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 6.717453956604004 | KNN Loss: 5.6381916999816895 | BCE Loss: 1.0792620182037354
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 6.6683526039123535 | KNN Loss: 5.634644031524658 | BCE Loss: 1.0337084531784058
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.681865692138672 | KNN Loss: 5.645390033721924 | BCE Loss: 1.0364757776260376
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.676312446594238 | KNN Loss: 5.6179609298706055 | BCE Loss: 1.0583512783050537
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.69525146484375 | KNN Loss: 5.647629737854004 | BCE Loss: 1.047621488571167
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.663219451904297 | KNN Loss: 5.6334547996521 | BCE Loss: 1.0297648906707764
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.682117462158203 | KNN Loss: 5.639921188354492 | 

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.0258,  3.9439,  2.6377,  3.6324,  3.5231,  0.6968,  2.7194,  2.2294,
          2.3823,  2.0247,  2.2558,  2.2561,  0.7992,  1.9055,  1.3400,  1.4725,
          2.8682,  3.3029,  2.8738,  2.3719,  1.7438,  3.0170,  2.3850,  2.6840,
          2.6220,  1.7341,  2.1848,  1.4313,  1.5433,  0.3170, -0.2358,  0.9944,
          0.1910,  0.9046,  1.5290,  1.4944,  0.9864,  3.3969,  0.8239,  1.3225,
          0.9583, -0.7440, -0.2944,  2.3565,  2.2295,  0.7441, -0.2419,  0.1186,
          1.5079,  2.5429,  1.8524,  0.0965,  1.4037,  0.5762, -0.6752,  1.1513,
          1.4866,  1.3908,  1.3941,  1.9165,  0.5962,  0.8472,  0.1199,  1.7733,
          1.3448,  1.6887, -1.9134,  0.2970,  2.3554,  2.2118,  2.6246,  0.4108,
          1.3821,  2.5102,  2.0509,  1.3257,  0.2335,  0.7602,  0.2090,  1.6089,
          0.0213,  0.3772,  1.8677, -0.4050,  0.2225, -1.1053, -2.4161, -0.2930,
          0.5203, -1.8782,  0.4301, -0.1449, -0.6335, -0.9953,  0.5809,  1.2839,
         -0.7181, -0.7370,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 83.52it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 030 | Total loss: 9.618 | Reg loss: 0.012 | Tree loss: 9.618 | Accuracy: 0.000000 | 1.236 sec/iter
Epoch: 00 | Batch: 001 / 030 | Total loss: 9.605 | Reg loss: 0.011 | Tree loss: 9.605 | Accuracy: 0.000000 | 1.038 sec/iter
Epoch: 00 | Batch: 002 / 030 | Total loss: 9.592 | Reg loss: 0.010 | Tree loss: 9.592 | Accuracy: 0.000000 | 0.977 sec/iter
Epoch: 00 | Batch: 003 / 030 | Total loss: 9.579 | Reg loss: 0.010 | Tree loss: 9.579 | Accuracy: 0.000000 | 0.948 sec/iter
Epoch: 00 | Batch: 004 / 030 | Total loss: 9.566 | Reg loss: 0.009 | Tree loss: 9.566 | Accuracy: 0.000000 | 0.933 sec/iter
Epoch: 00 | Batch: 005 / 030 | Total loss: 9.554 | Reg loss: 0.009 | Tree loss: 9.554 | Accuracy: 0.009766 | 0.925 sec/iter
Epoch: 00 | Batch: 006 / 030 | Total loss: 9.542 | Reg loss: 0.009 | Tree loss: 9.542 | Accuracy: 0.052734 | 0.918 

Epoch: 02 | Batch: 001 / 030 | Total loss: 9.272 | Reg loss: 0.009 | Tree loss: 9.272 | Accuracy: 0.554688 | 0.908 sec/iter
Epoch: 02 | Batch: 002 / 030 | Total loss: 9.260 | Reg loss: 0.009 | Tree loss: 9.260 | Accuracy: 0.591797 | 0.907 sec/iter
Epoch: 02 | Batch: 003 / 030 | Total loss: 9.249 | Reg loss: 0.010 | Tree loss: 9.249 | Accuracy: 0.603516 | 0.907 sec/iter
Epoch: 02 | Batch: 004 / 030 | Total loss: 9.234 | Reg loss: 0.010 | Tree loss: 9.234 | Accuracy: 0.599609 | 0.906 sec/iter
Epoch: 02 | Batch: 005 / 030 | Total loss: 9.230 | Reg loss: 0.010 | Tree loss: 9.230 | Accuracy: 0.552734 | 0.906 sec/iter
Epoch: 02 | Batch: 006 / 030 | Total loss: 9.220 | Reg loss: 0.010 | Tree loss: 9.220 | Accuracy: 0.542969 | 0.906 sec/iter
Epoch: 02 | Batch: 007 / 030 | Total loss: 9.199 | Reg loss: 0.011 | Tree loss: 9.199 | Accuracy: 0.593750 | 0.905 sec/iter
Epoch: 02 | Batch: 008 / 030 | Total loss: 9.190 | Reg loss: 0.011 | Tree loss: 9.190 | Accuracy: 0.589844 | 0.905 sec/iter
Epoch: 0

Epoch: 04 | Batch: 003 / 030 | Total loss: 8.926 | Reg loss: 0.016 | Tree loss: 8.926 | Accuracy: 0.589844 | 0.907 sec/iter
Epoch: 04 | Batch: 004 / 030 | Total loss: 8.912 | Reg loss: 0.016 | Tree loss: 8.912 | Accuracy: 0.556641 | 0.906 sec/iter
Epoch: 04 | Batch: 005 / 030 | Total loss: 8.893 | Reg loss: 0.016 | Tree loss: 8.893 | Accuracy: 0.552734 | 0.906 sec/iter
Epoch: 04 | Batch: 006 / 030 | Total loss: 8.877 | Reg loss: 0.017 | Tree loss: 8.877 | Accuracy: 0.574219 | 0.906 sec/iter
Epoch: 04 | Batch: 007 / 030 | Total loss: 8.852 | Reg loss: 0.017 | Tree loss: 8.852 | Accuracy: 0.593750 | 0.906 sec/iter
Epoch: 04 | Batch: 008 / 030 | Total loss: 8.838 | Reg loss: 0.017 | Tree loss: 8.838 | Accuracy: 0.570312 | 0.906 sec/iter
Epoch: 04 | Batch: 009 / 030 | Total loss: 8.819 | Reg loss: 0.018 | Tree loss: 8.819 | Accuracy: 0.578125 | 0.906 sec/iter
Epoch: 04 | Batch: 010 / 030 | Total loss: 8.809 | Reg loss: 0.018 | Tree loss: 8.809 | Accuracy: 0.554688 | 0.905 sec/iter
Epoch: 0

Epoch: 06 | Batch: 005 / 030 | Total loss: 8.455 | Reg loss: 0.022 | Tree loss: 8.455 | Accuracy: 0.593750 | 0.907 sec/iter
Epoch: 06 | Batch: 006 / 030 | Total loss: 8.424 | Reg loss: 0.023 | Tree loss: 8.424 | Accuracy: 0.566406 | 0.906 sec/iter
Epoch: 06 | Batch: 007 / 030 | Total loss: 8.386 | Reg loss: 0.023 | Tree loss: 8.386 | Accuracy: 0.560547 | 0.906 sec/iter
Epoch: 06 | Batch: 008 / 030 | Total loss: 8.360 | Reg loss: 0.023 | Tree loss: 8.360 | Accuracy: 0.611328 | 0.906 sec/iter
Epoch: 06 | Batch: 009 / 030 | Total loss: 8.345 | Reg loss: 0.024 | Tree loss: 8.345 | Accuracy: 0.562500 | 0.906 sec/iter
Epoch: 06 | Batch: 010 / 030 | Total loss: 8.310 | Reg loss: 0.024 | Tree loss: 8.310 | Accuracy: 0.574219 | 0.906 sec/iter
Epoch: 06 | Batch: 011 / 030 | Total loss: 8.291 | Reg loss: 0.025 | Tree loss: 8.291 | Accuracy: 0.583984 | 0.906 sec/iter
Epoch: 06 | Batch: 012 / 030 | Total loss: 8.285 | Reg loss: 0.025 | Tree loss: 8.285 | Accuracy: 0.535156 | 0.906 sec/iter
Epoch: 0

Epoch: 08 | Batch: 007 / 030 | Total loss: 7.881 | Reg loss: 0.028 | Tree loss: 7.881 | Accuracy: 0.601562 | 0.907 sec/iter
Epoch: 08 | Batch: 008 / 030 | Total loss: 7.853 | Reg loss: 0.028 | Tree loss: 7.853 | Accuracy: 0.580078 | 0.907 sec/iter
Epoch: 08 | Batch: 009 / 030 | Total loss: 7.826 | Reg loss: 0.029 | Tree loss: 7.826 | Accuracy: 0.548828 | 0.907 sec/iter
Epoch: 08 | Batch: 010 / 030 | Total loss: 7.792 | Reg loss: 0.029 | Tree loss: 7.792 | Accuracy: 0.570312 | 0.907 sec/iter
Epoch: 08 | Batch: 011 / 030 | Total loss: 7.710 | Reg loss: 0.030 | Tree loss: 7.710 | Accuracy: 0.583984 | 0.907 sec/iter
Epoch: 08 | Batch: 012 / 030 | Total loss: 7.729 | Reg loss: 0.030 | Tree loss: 7.729 | Accuracy: 0.578125 | 0.906 sec/iter
Epoch: 08 | Batch: 013 / 030 | Total loss: 7.670 | Reg loss: 0.030 | Tree loss: 7.670 | Accuracy: 0.552734 | 0.906 sec/iter
Epoch: 08 | Batch: 014 / 030 | Total loss: 7.642 | Reg loss: 0.031 | Tree loss: 7.642 | Accuracy: 0.580078 | 0.906 sec/iter
Epoch: 0

...

Epoch: 90 | Batch: 024 / 030 | Total loss: 1.019 | Reg loss: 0.049 | Tree loss: 1.019 | Accuracy: 0.593750 | 0.907 sec/iter
Epoch: 90 | Batch: 025 / 030 | Total loss: 1.017 | Reg loss: 0.049 | Tree loss: 1.017 | Accuracy: 0.566406 | 0.907 sec/iter
Epoch: 90 | Batch: 026 / 030 | Total loss: 1.012 | Reg loss: 0.049 | Tree loss: 1.012 | Accuracy: 0.566406 | 0.907 sec/iter
Epoch: 90 | Batch: 027 / 030 | Total loss: 0.967 | Reg loss: 0.049 | Tree loss: 0.967 | Accuracy: 0.587891 | 0.907 sec/iter
Epoch: 90 | Batch: 028 / 030 | Total loss: 0.970 | Reg loss: 0.049 | Tree loss: 0.970 | Accuracy: 0.562500 | 0.907 sec/iter
Epoch: 90 | Batch: 029 / 030 | Total loss: 0.966 | Reg loss: 0.049 | Tree loss: 0.966 | Accuracy: 0.571429 | 0.907 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.98214285714

Epoch: 92 | Batch: 026 / 030 | Total loss: 0.975 | Reg loss: 0.049 | Tree loss: 0.975 | Accuracy: 0.595703 | 0.907 sec/iter
Epoch: 92 | Batch: 027 / 030 | Total loss: 0.984 | Reg loss: 0.049 | Tree loss: 0.984 | Accuracy: 0.568359 | 0.907 sec/iter
Epoch: 92 | Batch: 028 / 030 | Total loss: 0.953 | Reg loss: 0.049 | Tree loss: 0.953 | Accuracy: 0.607422 | 0.907 sec/iter
Epoch: 92 | Batch: 029 / 030 | Total loss: 0.929 | Reg loss: 0.049 | Tree loss: 0.929 | Accuracy: 0.580952 | 0.907 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 93 | Batch: 000 / 030 | Total loss: 1.690 | Reg loss: 0.046 | Tree loss: 1.690 | Accuracy: 0.582031 | 0.907 sec/iter
Epoch: 93 | Batch: 001 / 030 | Total loss: 1.675 | Reg loss: 0.046 | Tree loss: 1.675 | Ac

Epoch: 94 | Batch: 028 / 030 | Total loss: 0.965 | Reg loss: 0.049 | Tree loss: 0.965 | Accuracy: 0.580078 | 0.907 sec/iter
Epoch: 94 | Batch: 029 / 030 | Total loss: 0.962 | Reg loss: 0.049 | Tree loss: 0.962 | Accuracy: 0.561905 | 0.907 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 95 | Batch: 000 / 030 | Total loss: 1.688 | Reg loss: 0.046 | Tree loss: 1.688 | Accuracy: 0.583984 | 0.907 sec/iter
Epoch: 95 | Batch: 001 / 030 | Total loss: 1.598 | Reg loss: 0.046 | Tree loss: 1.598 | Accuracy: 0.572266 | 0.907 sec/iter
Epoch: 95 | Batch: 002 / 030 | Total loss: 1.625 | Reg loss: 0.046 | Tree loss: 1.625 | Accuracy: 0.562500 | 0.907 sec/iter
Epoch: 95 | Batch: 003 / 030 | Total loss: 1.628 | Reg loss: 0.046 | Tree loss: 1.628 | Ac

Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 97 | Batch: 000 / 030 | Total loss: 1.613 | Reg loss: 0.046 | Tree loss: 1.613 | Accuracy: 0.554688 | 0.906 sec/iter
Epoch: 97 | Batch: 001 / 030 | Total loss: 1.663 | Reg loss: 0.046 | Tree loss: 1.663 | Accuracy: 0.574219 | 0.906 sec/iter
Epoch: 97 | Batch: 002 / 030 | Total loss: 1.588 | Reg loss: 0.046 | Tree loss: 1.588 | Accuracy: 0.566406 | 0.906 sec/iter
Epoch: 97 | Batch: 003 / 030 | Total loss: 1.571 | Reg loss: 0.046 | Tree loss: 1.571 | Accuracy: 0.578125 | 0.906 sec/iter
Epoch: 97 | Batch: 004 / 030 | Total loss: 1.510 | Reg loss: 0.046 | Tree loss: 1.510 | Accuracy: 0.572266 | 0.906 sec/iter
Epoch: 97 | Batch: 005 / 030 | Total loss: 1.469 | Reg loss: 0.046 | Tree loss: 1.469 | Ac

Epoch: 99 | Batch: 000 / 030 | Total loss: 1.669 | Reg loss: 0.046 | Tree loss: 1.669 | Accuracy: 0.583984 | 0.906 sec/iter
Epoch: 99 | Batch: 001 / 030 | Total loss: 1.619 | Reg loss: 0.046 | Tree loss: 1.619 | Accuracy: 0.582031 | 0.906 sec/iter
Epoch: 99 | Batch: 002 / 030 | Total loss: 1.657 | Reg loss: 0.046 | Tree loss: 1.657 | Accuracy: 0.576172 | 0.906 sec/iter
Epoch: 99 | Batch: 003 / 030 | Total loss: 1.579 | Reg loss: 0.046 | Tree loss: 1.579 | Accuracy: 0.574219 | 0.906 sec/iter
Epoch: 99 | Batch: 004 / 030 | Total loss: 1.505 | Reg loss: 0.046 | Tree loss: 1.505 | Accuracy: 0.568359 | 0.906 sec/iter
Epoch: 99 | Batch: 005 / 030 | Total loss: 1.545 | Reg loss: 0.046 | Tree loss: 1.545 | Accuracy: 0.562500 | 0.906 sec/iter
Epoch: 99 | Batch: 006 / 030 | Total loss: 1.448 | Reg loss: 0.046 | Tree loss: 1.448 | Accuracy: 0.601562 | 0.906 sec/iter
Epoch: 99 | Batch: 007 / 030 | Total loss: 1.429 | Reg loss: 0.046 | Tree loss: 1.429 | Accuracy: 0.578125 | 0.906 sec/iter
Epoch: 9

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 9.191358024691358


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 324


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


9468
============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
3127
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== 